In [74]:
from datetime import datetime, timedelta, time
from pydantic import BaseModel, Field
from typing import Optional

input_text = "Monday at 12:00 PM"

1. We need to identify all date and time references in the input text
- This can be done through keyword inferences and patterns such as the existence of PM or AM, months, days of the week, prefixes like st, th, nd, rd, etc., or other vague keywords like today, tomorrow, next week, etc.
2. After doing this, we need to identify the length of each list, given that both must have the same length, and if one is less than the other, we make an implication that the other has the same date or time. With this in mind, we need to keep note that there must always occur a pair of two dates and times.
- For this to work, we need to ensure that we get the right datetimes that correspond with each other. This can be done by taking an inference of the order that the dates or times were placed in. For example, a date such as "tomorrow from 6pm to 7pm", we can infer that tomorrow at 6pm refers to a date that comes before 7pm because it was referenced before. This is true for more complex requests such as "From Monday from 5pm to 7pm to Tuesday from 5pm to 7pm." Here we see that there are two dates and four times, hwoever each date found has two times that follow it directly after. With this in mind we can infer that monday corresponds to 5pm to 7pm and tuesday is another date that corresponds to the time 5pm to 7pm as well.
3. After locating these datetimes, we will convert them to datetime objects for easier processing going further
4. Once we identify the correct start and end times that are being referenced, we need to now filter these times by checking if we are requesting just one date or two. This can be true for task requests that only take noe datetime as input. To do this, we fetch all the events and tasks that already exist for a user, and compare them against the datetimes we found in the input text. If we find a match, we will infer that the input text is not targetting this datetime as a datetime of interest.

In [44]:
class DateTimeSet(BaseModel):
    input_tokens: list[str] = Field(default=[], description="A list of all the input tokens found within an input text")
    times: list[time] = Field(default=[], description="A list of all the times found within an input text")
    dates: list[datetime] = Field(default=[], description="A list of all the dates found within an input text")
    datetimes: list[datetime] = Field(default=[], description="A list of all the datetime objects found within an input text")
    target_datetimes: list[tuple[datetime, None]] = Field(default=[], description="A list of all the target datetime objects found within an input text as tuples representing start and end or due and None")
    is_event: Optional[bool] = Field(default=False, description="Indicates whether the extracted datetimes are part of an event")


In [53]:
def fetch_days_ahead(target_day: int, current_day=datetime.now()):
    current_week_day = current_day.weekday()
    days_ahead = (target_day - current_week_day) % 7
    if days_ahead == 0:
        days_ahead = 7

    return current_day + timedelta(days=days_ahead)

def fetch_year_month(current_year: int, current_month: int, target_month: int):
    if target_month < current_month:
        return datetime(current_year + 1, target_month, 1).strftime('%y-%m-%d')
    return datetime(current_year, target_month, 1).strftime('%y-%m-%d')

date_keys = {
    "today": datetime.now().strftime('%y-%m-%d'),
    "tonight": datetime.now().strftime('%y-%m-%d'),
    "evening": datetime.now().strftime('%y-%m-%d'),
    "morning": datetime.now().strftime('%y-%m-%d'),
    "afternoon": datetime.now().strftime('%y-%m-%d'),
    "tomorrow": (datetime.now() + timedelta(days=1)).strftime('%y-%m-%d'),
    "monday": fetch_days_ahead(0).strftime('%y-%m-%d'),
    "tuesday": fetch_days_ahead(1).strftime('%y-%m-%d'),
    "wednesday": fetch_days_ahead(2).strftime('%y-%m-%d'),
    "thursday": fetch_days_ahead(3).strftime('%y-%m-%d'),
    "friday": fetch_days_ahead(4).strftime('%y-%m-%d'),
    "saturday": fetch_days_ahead(5).strftime('%y-%m-%d'),
    "sunday": fetch_days_ahead(6).strftime('%y-%m-%d'),
    "january": fetch_year_month(datetime.now().year, datetime.now().month, 1),
    "february": fetch_year_month(datetime.now().year, datetime.now().month, 2),
    "march": fetch_year_month(datetime.now().year, datetime.now().month, 3),
    "april": fetch_year_month(datetime.now().year, datetime.now().month, 4),
    "may": fetch_year_month(datetime.now().year, datetime.now().month, 5),
    "june": fetch_year_month(datetime.now().year, datetime.now().month, 6),
    "july": fetch_year_month(datetime.now().year, datetime.now().month, 7),
    "august": fetch_year_month(datetime.now().year, datetime.now().month, 8),
    "september": fetch_year_month(datetime.now().year, datetime.now().month, 9),
    "october": fetch_year_month(datetime.now().year, datetime.now().month, 10),
    "november": fetch_year_month(datetime.now().year, datetime.now().month, 11),
    "december": fetch_year_month(datetime.now().year, datetime.now().month, 12)
}

In [75]:
def compile_datetimes(input_text, datetime_set: DateTimeSet):
    datetime_set.input_tokens = input_text.split(" ")
    for i, token in enumerate(datetime_set.input_tokens):
        token = token.lower()
        if token in date_keys:
            datetime_set.dates.append(date_keys[token]) #type: ignore
        if ":" in token:
            hour, minute = token.split(":")
            if "12:" in token and "am" in datetime_set.input_tokens[i+1].lower():
                hour = 0
            if i < len(datetime_set.input_tokens) - 1 and "pm" in datetime_set.input_tokens[i+1].lower() and "12:" not in token:
                hour = int(hour) + 12
            datetime_set.times.append(time(int(hour), int(minute)))
    print(f"Found Times: {datetime_set.times}")
    print(f"Found Dates: {datetime_set.dates}")
    return datetime_set

def organize_for_datetimes(datetime_set: DateTimeSet):
    imply_dates = False
    imply_times = False
    if len(datetime_set.dates) < len(datetime_set.times):
        imply_dates = True
        if len(datetime_set.times) % len(datetime_set.dates) != 0:
            raise ValueError("The number of times must be a multiple of the number of dates to imply dates correctly.")
    elif len(datetime_set.dates) > len(datetime_set.times):
        imply_times = True
        if len(datetime_set.dates) % len(datetime_set.times) != 0:
            raise ValueError("The number of dates must be a multiple of the number of times to imply times correctly.")
    else:
        for date, time in zip(datetime_set.dates, datetime_set.times):
            datetime_set.datetimes.append(datetime.strptime(f"{date} {time}", '%y-%m-%d %H:%M:%S'))
        return datetime_set

    print(f"Implying dates: {imply_dates} \nImplying times: {imply_times}")
    if imply_dates:
        interval = len(datetime_set.times) // len(datetime_set.dates)
        for i in range(0, len(datetime_set.times), interval):
            for j in range(interval):
                date = datetime_set.dates[i // interval]
                time = datetime_set.times[i+j]
                datetime_set.datetimes.append(datetime.strptime(f"{date} {time}", '%y-%m-%d %H:%M:%S'))

    if imply_times:
        interval = len(datetime_set.dates) // len(datetime_set.times)
        for i in range(0, len(datetime_set.dates), interval):
            for j in range(interval):
                date = datetime_set.dates[i+j]
                time = datetime_set.times[i // interval]
                datetime_set.datetimes.append(datetime.strptime(f"{date} {time}", '%y-%m-%d %H:%M:%S'))
    return datetime_set

datetime_set = DateTimeSet()
datetime_set = compile_datetimes(input_text, datetime_set)
datetime_set = organize_for_datetimes(datetime_set)
if datetime_set:
    for dt in datetime_set:
        print(dt)


Found Times: [datetime.time(12, 0)]
Found Dates: ['25-09-08']
('input_tokens', ['Monday', 'at', '12:00', 'PM'])
('times', [datetime.time(12, 0)])
('dates', ['25-09-08'])
('datetimes', [datetime.datetime(2025, 9, 8, 12, 0)])
('target_datetimes', [])
('is_event', False)
